In [1]:
from functions import *

In [2]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import os
import rasterio
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pyproj
from geopandas.tools import sjoin
from shapely.geometry import Point
import requests
from scipy.stats import ks_2samp
from rasterio.transform import from_origin
import shutil

/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Load data

In [3]:
W = 256
source_path = '../data/raw/'
data_paths = ['lst', 'ndvi', 'wt', 'wt_interpolated']
dir_paths = [os.path.join(source_path,p) for p in data_paths]
list_rgb = [True,False,False,False]

In [4]:
data, time_slots = load_data(dir_paths,W,list_rgb)

/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/skimage/transform/_warps.py:738: RuntimeWarning: All-NaN slice encountered
  min_val = min_func(input_image)
/Users/ruthparajo/Library/Python/3.9/lib/python/site-packages/skimage/transform/_warps.py:742: RuntimeWarning: All-NaN slice encountered
  max_val = max_func(input_image)


In [5]:
data_inputs = data[dir_paths[0]]
ndvi = data[dir_paths[1]]
data_targets = data[dir_paths[2]]
print('inputs shape:', data_inputs.shape,ndvi.shape)
print('targets shape:', data_targets.shape)

inputs shape: (125, 256, 256, 3) (125, 256, 256)
targets shape: (108, 256, 256)


In [11]:
start_date = '2013-03'
end_date = '2023-07'

# Generar el rango de fechas en formato YYYY-MM
date_range = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m')

# Convertir a un array de numpy (opcional, si quieres un array en lugar de un pandas Index)
time_slots = date_range.to_numpy()
time_slots

array(['2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08',
       '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02',
       '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08',
       '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02',
       '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08',
       '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02',
       '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08',
       '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02',
       '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
       '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
       '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
       '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12', 

In [7]:
gdf_points = gpd.read_file('../data/external/wt_filtered/wt_filtered.geojson')
time_slots = gdf_points.time.dt.strftime("%Y-%m").unique()
months = gdf_points.time.dt.month.unique()

# Delete unuseful data

In [8]:
'''new_inputs = []
new_targets = []
new_masked_inputs = []
del_indices = []
for index in range(0,data_inputs.shape[0]):
  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])

  blacks = img_gray==0
  percentage_b = np.sum(blacks)/blacks.size * 100

  whites = img_gray>=240
  percentage_w = np.sum(whites)/whites.size * 100

  if not np.all(img_gray==255) and not np.all(img_gray==0) and not np.all(img_gray == img_gray[0,0]) and not (percentage_b > 50) and not (percentage_w > 50):
    new_inputs.append(data_inputs[index])
    new_targets.append(data_targets[index])
    new_masked_inputs.append(masked_inputs[index])
  else:
    del_indices.append(index)

new_inputs = np.array(new_inputs)
new_targets = np.array(new_targets)
new_masked_inputs = np.array(new_masked_inputs)
slots = np.delete(time_slots,del_indices)'''

'new_inputs = []\nnew_targets = []\nnew_masked_inputs = []\ndel_indices = []\nfor index in range(0,data_inputs.shape[0]):\n  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])\n\n  blacks = img_gray==0\n  percentage_b = np.sum(blacks)/blacks.size * 100\n\n  whites = img_gray>=240\n  percentage_w = np.sum(whites)/whites.size * 100\n\n  if not np.all(img_gray==255) and not np.all(img_gray==0) and not np.all(img_gray == img_gray[0,0]) and not (percentage_b > 50) and not (percentage_w > 50):\n    new_inputs.append(data_inputs[index])\n    new_targets.append(data_targets[index])\n    new_masked_inputs.append(masked_inputs[index])\n  else:\n    del_indices.append(index)\n\nnew_inputs = np.array(new_inputs)\nnew_targets = np.array(new_targets)\nnew_masked_inputs = np.array(new_masked_inputs)\nslots = np.delete(time_slots,del_indices)'

Including NDVI data

In [ ]:
del_indices = []
fixed_ndvi = []
for index in range(0,data_inputs.shape[0]):
  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])

  blacks = img_gray==0
  percentage_b = np.sum(blacks)/blacks.size * 100

  whites = img_gray>=240
  percentage_w = np.sum(whites)/whites.size * 100

  img_ndvi = ndvi[index]

  whites = np.isnan(img_ndvi)
  percentage_w_ndvi = np.sum(whites)/whites.size * 100

  if np.all(img_gray == 255) or np.all(img_gray == 0) or np.all(img_gray == img_gray[0,0]) or (percentage_b > 50) or (percentage_w > 50) \
    or np.all(np.isnan(img_ndvi)) or np.all(img_ndvi == img_ndvi[0,0]) or (percentage_w_ndvi > 50):
    del_indices.append(index)
  else:
    new_ndvi = np.where(whites, 0.0, img_ndvi) # process ndvi images to not have nans
    fixed_ndvi.append(new_ndvi)


In [39]:
fixed_ndvi = np.array(fixed_ndvi)

In [12]:
slots = np.delete(time_slots,del_indices)

In [13]:
len(slots)

34

# Mask Landsat data

In [14]:
# Selecciona la imagen en el índice 0 (la primera imagen)
river_mask = data_targets[0]  # Cambia el índice según sea necesario

# Imprimir la forma de la imagen seleccionada
print('Selected image shape:', river_mask.shape)

Selected image shape: (256, 256)


In [15]:
# Supongamos que river_mask es tu máscara de río de forma (height, width)
# y que quieres que tenga 3 canales (para las imágenes RGB de data_inputs)
river_mask_expanded = np.expand_dims(river_mask, axis=-1)  # Expande para tener forma (height, width, 1)
river_mask_expanded = np.repeat(river_mask_expanded, data_inputs.shape[-1], axis=-1)  # Expande a (height, width, 3)

# Aplica la máscara solo a las entradas (inputs)
masked_inputs = []

for i,input_img in enumerate(data_inputs):
  if i not in del_indices:
    masked_input = input_img * river_mask_expanded  # Aplica la máscara a los inputs
    masked_inputs.append(masked_input)

# Convierte de nuevo a arrays de numpy
masked_inputs = np.array(masked_inputs)

# Verifica las formas
print('masked inputs shape:', masked_inputs.shape)

masked inputs shape: (34, 256, 256, 3)


# Mask NDVI data

In [40]:
masked_inputs_ndvi = []

for i,input_img in enumerate(fixed_ndvi):
    masked_input_ndvi = input_img * river_mask  # Aplica la máscara a los inputs
    masked_inputs_ndvi.append(masked_input_ndvi)

# Convierte de nuevo a arrays de numpy
masked_inputs_ndvi = np.array(masked_inputs_ndvi)

# Verifica las formas
print('masked inputs shape:', masked_inputs_ndvi.shape)

masked inputs shape: (34, 256, 256)


# Download preprocessed data

In [17]:
boiron = gpd.read_file('../data/external/swissTLMboiron')

bounds = boiron.total_bounds
# Definir la resolución
resolution = 30
x_min, y_min, x_max, y_max = bounds

transform = from_origin(x_min, y_max, resolution, resolution)  # Origen en la esquina superior izquierda

In [18]:
# Function to clear the contents of a directory
def clear_directory(directory):
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Error deleting {file_path}. Reason: {e}')

In [19]:
def save_raster(raster_array,filepath,shp):
  resolution = 30
  x_min, y_min, x_max, y_max = shp.total_bounds
  transform = from_origin(x_min, y_max, resolution, resolution)

  with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=raster_array.shape[0],
        width=raster_array.shape[1],
        count=len(raster_array.shape),
        dtype=raster_array.dtype,
        crs=shp.crs.to_string(),  # Ensure correct CRS
        transform=transform,
        nodata=0.0
) as dst:
    if len(raster_array.shape) == 3:
      dst.write(raster_array[:, :, 0], 1)  # Red channel
      dst.write(raster_array[:, :, 1], 2)  # Green channel
      dst.write(raster_array[:, :, 2], 3)  # Blue channel
    else:
      dst.write(raster_array, 1)

Clean preprocessed directories

In [41]:
raw_data_path = '../data/raw/'
preprocessed_data_path = '../data/preprocessed/'

data = ['wt', 'lst', 'ndvi', 'wt_interpolated']#, 'lst/masked']
for d in data:
  dest_dir = os.path.join(preprocessed_data_path, d)
  os.makedirs(dest_dir, exist_ok=True)
  clear_directory(dest_dir)
  if d == 'lst':
    dest_dir_m = os.path.join(dest_dir, 'masked')
    os.makedirs(dest_dir_m, exist_ok=True)
    clear_directory(dest_dir_m)

Copy preprocessed images

In [ ]:
for d in data:
  source_dir = os.path.join(raw_data_path, d)
  dest_dir = os.path.join(preprocessed_data_path, d)

  # List all files in the source directory
  all_images = os.listdir(source_dir)
  count = 0
  # Loop through each file and check if it contains any of the slots
  for ind, image_file in enumerate(all_images):
      # Check if the image filename contains any of the time slots
      if any(slot in image_file for slot in slots):
          # Construct full file paths
          if d == 'ndvi':
            date = image_file.split('_')[1].split('.')[0]
            name = f'{dest_dir}/ndvi_{date}.tiff'
            print('soc index',ind)
            print(np.sum(np.isnan(fixed_ndvi[count])))
            save_raster(fixed_ndvi[count], name, boiron)
            plt.imshow(fixed_ndvi[count])
            plt.show()
            print(f'Copied: ndvi_{date}, at {dest_dir}')
          else:
            dst_path = os.path.join(dest_dir, image_file)
            src_path = os.path.join(source_dir, image_file)
            # Copy the file to the destination folder
            shutil.copy(src_path, dst_path)
            print()
            print('Image', count)
            print(f'Copied: {image_file}, from {src_path}, to {dst_path}')

            if d == 'lst':
              date = image_file.split('_')[1].split('.')[0]
              name = f'{dest_dir}/masked/lst_masked_{date}.tiff'
              save_raster(masked_inputs[count], name, boiron)
              print(f'Copied: lst_masked_{date}')

          count+=1




CHECK

In [43]:
dir_paths = ['../data/preprocessed/ndvi']
W = 256
list_rgb = [False]
data, time_slots = load_data(dir_paths,W,list_rgb)

In [ ]:
n = data[dir_paths[0]]
print(n.shape)
for i in n:
  print(np.sum(np.isnan(i)))
  plt.imshow(i)
  plt.show()

# Pixel wise dataset creation

In [45]:
river_indices = np.nonzero(river_mask.reshape(-1))[0]

In [46]:
len(river_indices)

4215

### LST dataset

In [47]:
pixel_ds_lst = pd.DataFrame(columns=[river_indices])
pixel_ds_lst

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [48]:
for i,image in enumerate(masked_inputs):
  img_gray = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140]) # turn to gray image, calculate the weighted average of each channel contribution

  flat_image = img_gray.flatten()

  river_image = flat_image[river_indices]
  pixel_ds_lst.loc[slots[i]] = river_image

In [49]:
pixel_ds_lst

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,43.159259,3.307900e+01,1.900531e+01,42.060504,21.975468,4.695571e-04,8.288998,32.228627,40.992362,13.346206,...,29.025641,5.389397,0.001053,0.000939,1.608773e-04,0.000819,7.373843,36.191042,44.093145,39.729152
2014-01,5.053365,3.788743e+00,2.074472e+00,4.599467,2.280873,4.739557e-05,0.800507,3.128993,3.870447,1.295748,...,6.098846,1.184289,0.000261,0.000272,3.658324e-05,0.000181,1.563796,7.967361,10.440027,10.743683
2014-09,71.353529,5.479312e+01,3.179433e+01,69.725097,36.351236,7.711347e-04,13.768504,53.607845,67.823974,22.157293,...,58.396967,10.577701,0.001948,0.001554,3.350420e-04,0.001668,14.927864,71.520545,83.342885,68.504653
2015-03,0.044703,2.768079e-07,1.773953e-08,0.331079,0.128353,3.009348e-07,0.080520,0.314734,0.953364,0.312042,...,0.000000,0.000000,0.000000,0.000000,5.532135e-13,0.000000,0.000000,0.000000,0.000000,0.000000
2016-07,22.754020,1.788222e+01,1.047773e+01,22.467641,11.909886,2.552932e-04,4.482838,17.522360,22.007800,7.203296,...,16.396308,3.017694,0.000584,0.000505,9.168055e-05,0.000462,4.127973,20.254345,24.251856,20.593576
2016-09,39.465324,3.031221e+01,1.756632e+01,38.245350,19.848703,4.239691e-04,7.488491,28.889406,36.945165,11.963400,...,27.138163,5.119658,0.001023,0.000947,1.496577e-04,0.000760,6.853432,34.116344,42.247980,39.303488
2016-10,2.349577,1.532386e+00,8.978706e-01,1.701855,0.850706,1.823523e-05,0.240152,0.938699,1.187647,0.388724,...,5.887016,1.122599,0.000238,0.000251,3.303963e-05,0.000169,1.506258,7.459229,9.911714,10.368762
2016-12,58.687841,4.576831e+01,2.658899e+01,58.116454,30.482530,6.502996e-04,11.527297,45.057496,56.845432,18.605880,...,46.599474,8.543006,0.001637,0.001355,2.642232e-04,0.001340,12.224526,60.631794,71.312403,58.695225
2017-01,55.089666,4.280906e+01,2.486743e+01,53.897715,28.285980,6.022612e-04,10.646740,41.615605,52.652329,17.268987,...,34.022554,6.389747,0.001270,0.001167,1.877778e-04,0.000957,8.693935,43.193250,53.098380,48.723000
2017-03,45.648504,3.565704e+01,2.088017e+01,44.775240,23.335850,4.987174e-04,8.768832,33.893960,42.810172,13.818591,...,16.154127,2.971541,0.000579,0.000521,9.367819e-05,0.000472,4.202899,20.530234,25.241562,22.534612


In [51]:
pixel_ds_lst.to_csv('../data/preprocessed/lst/leboiron_lst.csv')

### NDVI dataset

In [52]:
pixel_ds_ndvi = pd.DataFrame(columns=[river_indices])
pixel_ds_ndvi

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [53]:
for i,image in enumerate(masked_inputs_ndvi):
  flat_image = image.flatten()
  river_image = flat_image[river_indices]

  #print(flat_image.shape, river_image.shape)

  pixel_ds_ndvi.loc[slots[i]] = river_image

In [54]:
pixel_ds_ndvi

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,0.680547,0.529035,0.303262,0.602555,0.331393,0.000008,0.107660,0.426750,0.574318,0.201110,...,0.344827,0.076673,0.000016,0.000015,2.455284e-06,1.237119e-05,0.105141,0.420968,0.439700,0.465491
2014-01,0.722355,0.485690,0.268500,0.618387,0.318958,0.000007,0.120549,0.459901,0.584997,0.186228,...,0.335747,0.083580,0.000016,0.000012,2.328145e-06,1.179892e-05,0.090164,0.462439,0.498935,0.503923
2014-09,1.081917,0.848812,0.483977,1.068012,0.566542,0.000012,0.203839,0.821204,1.041883,0.342191,...,0.393386,0.084204,0.000018,0.000016,2.322435e-06,1.171691e-05,0.094147,0.458299,0.498285,0.477993
2015-03,0.230682,0.174735,0.097345,0.228816,0.130479,0.000003,0.038288,0.196531,0.229880,0.083950,...,0.058852,0.011739,0.000003,0.000002,4.172990e-07,1.739308e-06,0.013626,0.070782,0.085840,0.079097
2016-07,0.605135,0.470862,0.280923,0.550168,0.305442,0.000007,0.097390,0.419717,0.506992,0.171966,...,0.293412,0.070291,0.000015,0.000011,1.906895e-06,9.676531e-06,0.077693,0.353523,0.383658,0.319256
2016-09,1.125991,0.875582,0.507404,1.111149,0.587925,0.000013,0.222782,0.870359,1.094016,0.354247,...,0.415045,0.085786,0.000019,0.000016,2.663394e-06,1.343586e-05,0.108758,0.460705,0.488192,0.471444
2016-10,0.553098,0.386566,0.225621,0.523765,0.249093,0.000006,0.096119,0.356929,0.396695,0.148172,...,0.255397,0.065545,0.000012,0.000009,1.930824e-06,9.404409e-06,0.071516,0.344343,0.378336,0.393361
2016-12,1.028744,0.791500,0.467677,1.013187,0.535860,0.000011,0.200324,0.791213,0.979794,0.322371,...,0.374538,0.081634,0.000017,0.000015,2.463141e-06,1.232025e-05,0.102938,0.427939,0.465669,0.440127
2017-01,0.156536,0.124441,0.071604,0.155531,0.081802,0.000002,0.030561,0.121385,0.144282,0.046022,...,0.035220,0.006384,0.000001,0.000001,1.912834e-07,9.958094e-07,0.008903,0.042000,0.052129,0.048460
2017-03,0.518296,0.395319,0.215170,0.483643,0.252923,0.000006,0.088332,0.378587,0.438885,0.148062,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000


In [55]:
pixel_ds_ndvi.to_csv('../data/preprocessed/ndvi/leboiron_ndvi.csv')

### Water temperature dataset

In [56]:
pixel_ds_wt = pd.DataFrame(columns=[river_indices])
pixel_ds_wt

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [57]:
c = 0
for i in range(len(data_targets)):
  if i not in del_indices:
    image = data_targets[i]
    flat_image = image.flatten()
    river_image = flat_image[river_indices]
    pixel_ds_wt.loc[slots[c]] = river_image
    c += 1

In [58]:
pixel_ds_wt

,29,30,31,285,286,287,541,542,797,798,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,14.045929,11.038595,6.467843,13.951374,7.351907,0.000158,2.767231,10.816454,13.685048,4.479206,...,6.935930,1.321777,0.000269,0.000254,0.000037,0.000190,1.743809,8.782688,11.038451,10.434786
2014-01,7.101403,5.580942,3.270041,7.053597,3.717010,0.000080,1.399069,5.468631,6.918947,2.264616,...,3.506698,0.668270,0.000136,0.000129,0.000019,0.000096,0.881643,4.440390,5.580869,5.275665
2014-09,18.768440,14.749984,8.642456,18.642094,9.823758,0.000211,3.697627,14.453154,18.286222,5.985200,...,9.267923,1.766184,0.000359,0.000340,0.000050,0.000254,2.330111,11.735597,14.749791,13.943162
2015-03,1.565705,1.230477,0.720973,1.555165,0.819520,0.000018,0.308464,1.205714,1.525478,0.499299,...,0.773151,0.147339,0.000030,0.000028,0.000004,0.000021,0.194383,0.979010,1.230461,1.163170
2016-07,13.850665,10.885139,6.377928,13.757424,7.249702,0.000155,2.728761,10.666086,13.494801,4.416936,...,6.839509,1.303402,0.000265,0.000251,0.000037,0.000188,1.719567,8.660593,10.884997,10.289724
2016-09,17.807108,13.994480,8.199783,17.687233,9.320580,0.000200,3.508232,13.712853,17.349590,5.678634,...,8.793214,1.675719,0.000341,0.000322,0.000047,0.000241,2.210761,11.134491,13.994297,13.228984
2016-10,17.355047,13.639208,7.991619,17.238214,9.083961,0.000195,3.419170,13.364732,16.909143,5.534473,...,8.569984,1.633178,0.000332,0.000314,0.000046,0.000235,2.154637,10.851825,13.639030,12.893146
2016-12,13.126761,10.316228,6.044586,13.038393,6.870797,0.000147,2.586143,10.108624,12.789495,4.186085,...,6.482042,1.235280,0.000251,0.000238,0.000035,0.000178,1.629694,8.207948,10.316093,9.751932
2017-01,7.708155,6.057784,3.549437,7.656265,4.034595,0.000086,1.518607,5.935877,7.510110,2.458108,...,3.806314,0.725368,0.000148,0.000140,0.000020,0.000104,0.956971,4.819782,6.057705,5.726424
2017-03,1.658442,1.303358,0.763676,1.647277,0.868060,0.000019,0.326735,1.277129,1.615832,0.528872,...,0.818945,0.156066,0.000032,0.000030,0.000004,0.000022,0.205896,1.036996,1.303341,1.232064


In [59]:
pixel_ds_wt.to_csv('../data/preprocessed/wt_interpolated/leboiron_wt_interpolated.csv')